In [1]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,accuracy_score,confusion_matrix
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge

In [2]:
main_dataset= pickle.load(open('dataset/main_dataset.pkl','rb'))
main_dataset.head()

,call_id,agent_id_x,agent_tone,customer_tone,average_sentiment,silence_percent_average,customer_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,...,elite_level_code,handle_time,allocation_time,hour,agent_tone_score,customer_tone_score,transcript_length,cleaned_transcript,silence_average_by_call_reason,call_time
0,4667960400,963118,neutral,angry,-0.04,0.39,2033123310,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,...,4.0,1860.0,420.0,23,3,1,2296,agent thank calling united airline customer se...,0.281767,23:56:00
1,1122072124,519057,calm,neutral,0.02,0.35,8186702651,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,...,0.0,720.0,180.0,0,4,3,2680,agent thank calling united airline name sam he...,0.260948,00:03:00
2,6834291559,158319,neutral,polite,-0.13,0.32,2416856629,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,...,0.0,1140.0,480.0,23,3,5,3128,agent thank calling united airline customer se...,0.309404,23:59:00
3,2266439882,488324,neutral,frustrated,-0.20,0.20,1154544516,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,...,2.0,420.0,300.0,0,3,2,3289,agent thank calling united airline customer se...,0.277023,00:05:00
4,1211603231,721730,neutral,polite,-0.05,0.35,5214456437,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,...,0.0,540.0,600.0,0,3,5,2820,agent thank calling united airline customer se...,0.264613,00:04:00


In [3]:
agent_data= pickle.load(open('dataset/agent_data.pkl','rb'))
agent_data.head()

,agent_id_x,total_calls,avg_tone_rating,avg_handle_time,avg_silence_percentage,avg_call_sentiment_score
0,102574,2,3.500000,3600.000000,0.260000,-0.075000
1,103651,298,3.278523,747.382550,0.293389,-0.027953
2,106000,113,3.292035,689.203540,0.274956,-0.032124
3,107876,6,3.166667,990.000000,0.405000,-0.065000
4,112609,11,3.454545,850.909091,0.286364,-0.020909


In [4]:
# # Encoding categorical variables
# # Dummy encoding for 'primary_call_reason'
# main_dataset = pd.get_dummies(main_dataset, columns=['primary_call_reason','customer_tone'], drop_first=True)

# # Check the new column names
# print(main_dataset.columns)

# # Define features and target, replace 
# # 'primary_call_reason' with its dummy variable columns
# X = main_dataset[['elite_level_code', 'hour'] + [col for col in main_dataset.columns if col.startswith('primary_call_reason_')]+[col1 for col1 in main_dataset.columns if col1.startswith('customer_tone_')]]  # Include all dummy columns for primary_call_reason
# y = main_dataset['handle_time']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create and train the model
# random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
# gradient_boosting = GradientBoostingRegressor(n_estimators=100, random_state=42)
# xgboost = XGBRegressor(n_estimators=100, random_state=42)

# stacking_model = StackingRegressor(
#     estimators=[
#         ('rf', random_forest),
#         ('gb', gradient_boosting),
#         ('xgb', xgboost)
#     ],
#     final_estimator=Ridge()
# )

# stacking_model.fit(X_train, y_train)

# # Predictions and evaluation
# y_pred = stacking_model.predict(X_test)
# print("Prediction: ", y_pred)
# print("Actual: ", y_test.values)

# # Calculate Mean Squared Error and Root Mean Squared Error
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# print(f'Root Mean Squared Error: {rmse}')
# print(f'Mean Squared Error: {mse}')

# # Create a DataFrame to compare actual and predicted values
# comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# # Display the DataFrame
# print("-------------------------------")
# print( comparison_df)


# # Feature importance (optional)
# # importances = model.feature_importances_
# # indices = importances.argsort()[::-1]

# # Display feature importance
# # for f in range(X.shape[1]):
# #     print(f'{f + 1}. Feature {X.columns[indices[f]]} ({importances[indices[f]]})')


In [5]:
main_dataset= pd.merge(main_dataset,agent_data,left_on=('agent_id_x'), right_on=('agent_id_x'), how=('left'))
main_dataset.rename(columns={'avg_silence_percentage': 'avg_silence_percentage_of_agent'}, inplace=True)
main_dataset.rename(columns={'avg_call_sentiment_score': 'avg_call_sentiment_score_of_agent'}, inplace=True)
main_dataset.head()

,call_id,agent_id_x,agent_tone,customer_tone,average_sentiment,silence_percent_average,customer_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,...,customer_tone_score,transcript_length,cleaned_transcript,silence_average_by_call_reason,call_time,total_calls,avg_tone_rating,avg_handle_time,avg_silence_percentage_of_agent,avg_call_sentiment_score_of_agent
0,4667960400,963118,neutral,angry,-0.04,0.39,2033123310,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,...,1,2296,agent thank calling united airline customer se...,0.281767,23:56:00,471,3.220807,649.808917,0.279427,-0.050114
1,1122072124,519057,calm,neutral,0.02,0.35,8186702651,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,...,3,2680,agent thank calling united airline name sam he...,0.260948,00:03:00,430,3.241860,671.860465,0.295907,-0.039404
2,6834291559,158319,neutral,polite,-0.13,0.32,2416856629,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,...,5,3128,agent thank calling united airline customer se...,0.309404,23:59:00,393,3.229008,681.374046,0.280153,-0.040645
3,2266439882,488324,neutral,frustrated,-0.20,0.20,1154544516,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,...,2,3289,agent thank calling united airline customer se...,0.277023,00:05:00,392,3.331633,611.785714,0.283316,-0.021939
4,1211603231,721730,neutral,polite,-0.05,0.35,5214456437,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,...,5,2820,agent thank calling united airline customer se...,0.264613,00:04:00,328,3.250000,649.756098,0.272957,-0.039817


In [6]:
# Check if there are duplicate column names in the dataset
print(main_dataset.columns[main_dataset.columns.duplicated()])


Index([], dtype='object')


In [7]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Step 1: Data Preparation
features = main_dataset[['agent_tone', 'customer_tone', 'avg_call_sentiment_score_of_agent', 'silence_average_by_call_reason' ,'primary_call_reason', 'hour']]
target = main_dataset['handle_time']

# Categorical and numerical features
categorical_features = ['agent_tone', 'customer_tone', 'primary_call_reason']
numerical_features = ['avg_call_sentiment_score_of_agent', 'silence_average_by_call_reason', 'hour']

# Preprocessor for categorical and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Step 2: Train the Model
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Pipeline for preprocessing and model
model_pipeline1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42))
])

# Fit the model with a sample of the data
model_pipeline1.fit(X_train, y_train)

# Make predictions
y_pred = model_pipeline1.predict(X_test)

# Step 3: Evaluate the Model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'R^2 Score: {r2}')

# Optionally, display predictions vs actual values
import pandas as pd
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results['delta']=  results['Actual']- results['Predicted']
print(results.head())



Mean Absolute Error: 555.641118207405
R^2 Score: -0.11822244399188486
       Actual  Predicted   delta
40896   600.0     395.20  204.80
61208   600.0     944.12 -344.12
38089   480.0    1381.00 -901.00
63460   840.0    1493.30 -653.30
24712   180.0     564.68 -384.68


In [8]:
sorted_results = results.sort_values(by='delta')
print(len(sorted_results))
count_above_250 = (results['delta'].abs() < 250).sum()
count_above_250

21543


7259

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer

# Step 1: Data Preparation (log-transform handle_time and categorize hour)
main_dataset['log_handle_time'] = np.log1p(main_dataset['handle_time'])  # Log-transform target
main_dataset['time_of_day'] = pd.cut(main_dataset['hour'], bins=[0, 12, 18, 24], labels=['morning', 'afternoon', 'evening'], right=False)

features = main_dataset[['agent_tone', 'customer_tone', 'avg_call_sentiment_score_of_agent', 'avg_silence_percentage_of_agent', 'primary_call_reason', 'time_of_day']]
target = main_dataset['log_handle_time']

# Categorical and numerical features
categorical_features = ['agent_tone', 'customer_tone', 'primary_call_reason', 'time_of_day']
numerical_features = ['avg_call_sentiment_score_of_agent', 'avg_silence_percentage_of_agent']

# Preprocessor: handle categorical with OneHotEncoder and passthrough numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_features),  # Impute missing values in numerical
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Handle categorical features
    ]
)

# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Step 3: Model Setup - XGBoost Regressor inside a Pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, n_jobs=-1))
])

# Step 4: Hyperparameter Tuning using RandomizedSearchCV
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [4, 5, 6],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__subsample': [0.8, 1.0],
    'regressor__colsample_bytree': [0.8, 1.0]
}

random_search = RandomizedSearchCV(model_pipeline, param_distributions=param_grid, n_iter=10, 
                                   scoring='neg_mean_absolute_error', cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Step 5: Model Evaluation
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# Convert predictions back to the original scale
y_pred_original = np.expm1(y_pred)  # Inverse log transformation
y_test_original = np.expm1(y_test)

# Step 6: Calculate performance metrics
mae = mean_absolute_error(y_test_original, y_pred_original)
r2 = r2_score(y_test_original, y_pred_original)

print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

# Optionally, display predictions vs actual values
results = pd.DataFrame({'Actual': y_test_original, 'Predicted': y_pred_original})
results['delta'] = results['Actual'] - results['Predicted']
print(results.head())

c:\Users\Ramneek\anaconda3\envs\MachineLearning\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
22 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Ramneek\anaconda3\envs\MachineLearning\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Ramneek\anaconda3\envs\MachineLearning\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Ramneek\anaconda3\envs\MachineLearning\lib\site-packages\sklearn\pipeline.py", line 

Mean Absolute Error: 463.0317034305694
R² Score: -0.09481388894085008
       Actual   Predicted       delta
40896   600.0  344.194366  255.805634
61208   600.0  475.595856  124.404144
38089   480.0  540.346619  -60.346619
63460   840.0  541.296875  298.703125
24712   180.0  411.667664 -231.667664


In [10]:
sorted_results = results.sort_values(by='delta')
print(len(sorted_results))
count_above_250 = (results['delta'].abs() <250).sum()
count_above_250

14362


7544

In [11]:
import joblib
joblib.dump(model_pipeline, 'dataset/model_pipeline.pkl')

['dataset/model_pipeline.pkl']